In [5]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print(gpus[0])

# Assume that you have 12GB of GPU memory and want to allocate ~4GB:
gpu_options = tf.config.experimental.set_memory_growth(gpus[0], True)



PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [2]:

import os
import json
import shutil

import numpy as np
from PIL import Image
from alive_progress import alive_bar
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from matplotlib import pyplot as plt

from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, Flatten, Dense, Activation, MaxPooling2D, Dropout, Input, BatchNormalization


import os
import json
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split


# tensorboard --logdir logs/


Image.MAX_IMAGE_PIXELS = None
directory = 'data/gba_data'
# with alive_bar(25*3, title="generating label/image batches", length=100, spinner="arrow") as bar:
#     for image in os.listdir(directory):
#         for i in range(3): #change here to get to lowest number that fits on GPU memory
#
#             f = os.path.join(directory, image)
#             img = Image.open(f)
#             data = np.asarray(img)
#
#             new_img = img.resize((round(data.shape[0]/2), round(data.shape[1]/2)))
#             bar()
#
#         print(data.shape)
#         img.save('data/training data/resized/'+ new_img)
#         # print(image)

# img.thumbnail((round(data.shape[0]/2), round(data.shape[1]/2)))
# img.save('image_12.jpg')
#
# data = np.asarray(img)

with alive_bar(25, title="generating label/image batches", length=100, spinner="arrow") as bar:
    for image in os.listdir(directory):
        f = os.path.join(directory, image)
        img = Image.open(f)
        data = np.asarray(img)
        # new_img = img.resize((round(data.shape[0]/10), round(data.shape[1]/10)))
        new_img = img.resize((1805, 1510))
        new_img.save('data/resized/' + image)
        new_data = np.asarray(new_img)
        print(new_data.shape)
        bar()


tf.config.experimental.list_physical_devices()

on 0: (1510, 1805, 3)
on 1: (1510, 1805, 3)
on 2: (1510, 1805, 3)
on 3: (1510, 1805, 3)
on 4: (1510, 1805, 3)
on 5: (1510, 1805, 3)
on 6: (1510, 1805, 3)
on 7: (1510, 1805, 3)
on 8: (1510, 1805, 3)
on 9: (1510, 1805, 3)
on 10: (1510, 1805, 3)
on 11: (1510, 1805, 3)
on 12: (1510, 1805, 3)
on 13: (1510, 1805, 3)
on 14: (1510, 1805, 3)
on 15: (1510, 1805, 3)
on 16: (1510, 1805, 3)
on 17: (1510, 1805, 3)
on 18: (1510, 1805, 3)
on 19: (1510, 1805, 3)
on 20: (1510, 1805, 3)
on 21: (1510, 1805, 3)
on 22: (1510, 1805, 3)
on 23: (1510, 1805, 3)
on 24: (1510, 1805, 3)
on 25: (1510, 1805, 3)
on 26: (1510, 1805, 3)
on 27: (1510, 1805, 3)
on 28: (1510, 1805, 3)
on 29: (1510, 1805, 3)
on 30: (1510, 1805, 3)
on 31: (1510, 1805, 3)
on 32: (1510, 1805, 3)
on 33: (1510, 1805, 3)
on 34: (1510, 1805, 3)
on 35: (1510, 1805, 3)
on 36: (1510, 1805, 3)
on 37: (1510, 1805, 3)
on 38: (1510, 1805, 3)
on 39: (1510, 1805, 3)
on 40: (1510, 1805, 3)
on 41: (1510, 1805, 3)
on 42: (1510, 1805, 3)
on 43: (1510, 1805, 3

NameError: name 'tf' is not defined

In [3]:
 # %reset -f


print(new_data.shape)
folder = 'logs/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

with tf.device('/GPU:0'):
    model = Sequential()

    model.add(Conv2D(50, kernel_size=5, padding='same', strides=2, activation='relu', input_shape=new_data.shape))
    model.add(Dropout(0.3))

    model.add(Conv2D(100, padding='same', kernel_size=3, strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(150, kernel_size=3, strides=2, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Conv2D(200, kernel_size=3, strides=2, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Conv2D(200, kernel_size=3, strides=2, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2, activation='linear'))

    model.compile(loss='mse',optimizer='adam', metrics=['mae'])

    checkpoint = ModelCheckpoint("rotnet_reg_model", monitor='mae', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)
    early = EarlyStopping(monitor='mae', min_delta=0.001, patience=20, verbose=2, mode='auto')

    tensorboard = TensorBoard()

    model.summary()
    # model.fit(x, y, shuffle=True, workers=1, max_queue_size=1, use_multiprocessing=True, batch_size=64, epochs=50, validation_data=(x_val, y_val), callbacks=[checkpoint, early, tensorboard])



(1510, 1805, 3)


NameError: name 'tf' is not defined